In [61]:
!wget https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz
!wget https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json
!tar -xvzf resnet50v2.tar.gz --warning=no-unknown-keyword

--2024-01-25 13:31:05--  https://s3.amazonaws.com/onnx-model-zoo/resnet/resnet50v2/resnet50v2.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.107.70, 52.216.248.70, 52.217.4.126, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.107.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96452466 (92M) [application/x-gzip]
Saving to: ‘resnet50v2.tar.gz’

resnet50v2.tar.gz   100%[===================>]  91.98M  4.49MB/s    in 20s     

2024-01-25 13:31:26 (4.60 MB/s) - ‘resnet50v2.tar.gz’ saved [96452466/96452466]

--2024-01-25 13:31:26--  https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14209 (14K) [text/plain]

In [2]:
import json
import numpy as np
import onnxruntime
from PIL import Image


In [3]:
def preprocess(input_data):
    # convert the input data into the float32 input
    img_data = input_data.astype('float32')

    #normalize
    mean_vec = np.array([0.485, 0.456, 0.406])
    stddev_vec = np.array([0.229, 0.224, 0.225])
    norm_img_data = np.zeros(img_data.shape).astype('float32')
    for i in range(img_data.shape[0]):
        norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]

    #add batch channel
    norm_img_data = norm_img_data.reshape(1, 3, 224, 224).astype('float32') 
    return norm_img_data

In [4]:
image = Image.open('car.jpg').resize((224, 224))
image_data = np.array(image).transpose(2, 0, 1).astype('float32')
image_data = np.expand_dims(image_data, axis=0)
print(image_data.shape)

(1, 3, 224, 224)


In [6]:
onnx_path = 'resnet50v2/resnet50v2.onnx'
session = onnxruntime.InferenceSession(onnx_path)

In [13]:
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
result = session.run([output_name], {input_name: image_data})[0]

In [8]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [14]:
p=softmax(result[0]).tolist()
with open('imagenet-simple-labels.json') as f:
    labels = json.load(f)
    arr = np.array(p).argsort()[-5:][::-1]
    for id in arr:
        print(labels[id], p[id])

German Shepherd Dog 0.9648066759109497
Australian Kelpie 0.008698190562427044
Malinois 0.008583959192037582
muzzle 0.0026328619569540024
Cardigan Welsh Corgi 0.0014788524713367224
